In [1]:
import numpy as np
from glob import glob
import torch
import torchvision
from torch.autograd import Variable
from torchvision import datasets
from torchvision import transforms
import torchvision.models as models
import cv2
import matplotlib.pyplot as plt
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
#load images
dog_files = np.array(glob("dogImages/*/*/*"))
human_files = np.array(glob("lfw/*/*"))

# print number of images in each dataset
print('Total dog images are %d ' % len(dog_files))
print('Total human images are %d ' % len(human_files))

Total dog images are 8351 
Total human images are 13233 


In [16]:
#face detector function
face_cascade = cv2.CascadeClassifier('detector_architectures/haarcascade_frontalface_default.xml')

def face_detector(img_path):
    
    image = cv2.imread(img_path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(image_gray)
    
    return len(faces) > 0


In [17]:
#calculate percentage of human in both files

human_files_per = human_files[:100]
dog_files_per = dog_files[:100]

sum_human_hd = 0
sum_dog_hd = 0

for i in range(0,len(human_files_per)):
    human_hd = human_files_per[i]
    dog_hd = dog_files_per[i]
    
    if face_detector(human_hd) == True:
        sum_human_hd += 1
    if face_detector(dog_hd) == True:
        sum_dog_hd += 1

per_human_hd = (sum_human_hd/len(human_files_per))*100
per_dog_hd = (sum_dog_hd/len(dog_files_per))*100
print("The percentage of human in human files is " ,per_human_hd)
print("The percentage of dog in dog files is " ,per_dog_hd)

The percentage of human in human files is  100.0
The percentage of dog in dog files is  52.0


In [3]:
#percentage of dog in both files
#dog detetctor using vgg16 model
from PIL import Image
VGG = models.vgg16(pretrained = True)

#VGG function to return index of images from ImageNet
def VGG_16(img):
#def VGG16_predict(img):
    image = Image.open(img)
    data_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])
    image_trans = data_transform(image)
    imagee = image_trans.unsqueeze(0)
    image_var = Variable(imagee)
    image_vgg = VGG(image_var)
    image_index = image_vgg.data.numpy().argmax()
    
    return image_index

In [4]:
#dog detector index b/w 151 and 268
def dog_detector(img):
    index_re = VGG_16(img)
    
    if index_re >= 151 and index_re <= 268:
        return True
    else:
        return False

In [5]:
#calculate percentage of dog in both files
human_files_per = human_files[:100]
dog_files_per = dog_files[:100]

sum_human_dd = 0
sum_dog_dd = 0

for i in range(0,len(human_files_per)):
    human_dd = human_files_per[i]
    dog_dd = dog_files_per[i]
    
   # d = dog_detector(dog_dd)
    
    if dog_detector(human_dd) == True:
        sum_human_dd += 1
    if dog_detector(dog_dd) == True:
        sum_dog_dd += 1

per_human_dd = (sum_human_dd/len(human_files_per))*100
per_dog_dd = (sum_dog_dd/len(dog_files_per))*100
print("The percentage of human in human files is " ,per_human_dd)
print("The percentage of dog in dog files is " ,per_dog_dd)

The percentage of human in human files is  1.0
The percentage of dog in dog files is  92.0


In [6]:
#CNN
#data preperation
data_train = os.path.join("dogImages/","train/")
data_valid = os.path.join("dogImages/","valid/")
data_test = os.path.join("dogImages/","test/")

#transforms

data_transform = { 
                    'train' : transforms.Compose([transforms.RandomResizedCrop(224),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])]),
    
                    'valid' : transforms.Compose([transforms.RandomResizedCrop(224),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])]),
    
                    'test' : transforms.Compose([transforms.RandomResizedCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])
                 }

#load image data set
train_set = datasets.ImageFolder(data_train, transform = data_transform['train'])
valid_set = datasets.ImageFolder(data_valid, transform = data_transform['valid'])
test_set = datasets.ImageFolder(data_test, transform = data_transform['test'])

#Dataloader
batch_size = 10
num_workers = 0
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,num_workers = num_workers,shuffle = True)
valid_loader = torch.utils.data.DataLoader(valid_set,batch_size = batch_size,num_workers = num_workers,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,num_workers = num_workers,shuffle = True)

loaders_scratch = {
    'train': train_loader,
    'valid': valid_loader,
    'test': test_loader
}

In [7]:
#CNN model architecture
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        self.conv1 = nn.Conv2d(3, 32, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)

        # pool
        self.pool = nn.MaxPool2d(2, 2)
        
        # fully-connected
        self.fc1 = nn.Linear(7*7*128, 500)
        self.fc2 = nn.Linear(500, 133) 
        
        # drop-out
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        ## Define forward behavior
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        
        # flatten
        x = x.view(-1, 7*7*128)
        
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        
        x = self.dropout(x)
        x = self.fc2(x)
        return x
         
# instantiate the CNN
model_scratch = Net()
print(model_scratch)

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=6272, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=133, bias=True)
  (dropout): Dropout(p=0.3)
)


In [8]:
#loss function and optimizer
#criterion = nn.MSELoss()
#optimizer = optim.Adam(net.parameters(), lr = 0.0001)
criterion_scratch = nn.CrossEntropyLoss()
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr = 0.05)

In [11]:
#train and validate model
#def train(loader, model, criterion, optimizer, n_epochs):
def train(n_epochs, loaders, model, optimizer, criterion):
    
    for epoch in range(1,n_epochs):
        train_loss = 0
        valid_loss = 0
    
        #train model 
        model.train()
        for batch_idx, (data,target) in enumerate(loaders['train']):
            
            # zero the parameter (weight) gradients
            optimizer.zero_grad()
            
            # forward pass to get outputs
            output = model(data)
            
            # calculate the loss
            loss = criterion(output, target)
            
            # backward pass to calculate the parameter gradients
            loss.backward()
            
            # update the parameters
            optimizer.step()
            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            if batch_idx % 100 == 0:
                print("Epoch: {}, Batch:{}, Training Loss: {:.5f}".format(epoch, batch_idx, train_loss))
                
        #validate model 
        model.eval()
        for batch_idx, (data,target) in enumerate(loaders['valid']):
            
            # forward pass to get outputs
            output = model(data)
            
            # calculate the loss
            loss = criterion(output, target)
  
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
            if batch_idx % 100 == 0:
                print("Epoch: {}, Batch:{}, Validation Loss: {:.5f}".format(epoch, batch_idx, valid_loss)) 
        '''       
        if valid_loss < valid_loss_min:
                torch.save(model.state_dict(), save_path)
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
                valid_loss_min = valid_loss
        '''   
    # return trained model
    return model       

In [12]:
#training the model
model_scratch = train(5, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch)

Epoch: 1, Batch:0, Training Loss: 4.88853


UnboundLocalError: local variable 'photoshop' referenced before assignment

In [ ]:
# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('saved_models/model_scratch.pt'))

In [ ]:
#test model
def test(loaders, model, criterion):

    # monitor test loss and accuracy
    test_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        #if use_cuda:
         #   data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, model_scratch, criterion_scratch)